## Slicing NAM model for SPECFEM3D_Cart

---

In [ ]:
from sys import argv
import numpy as np
import pandas as pd
import scipy as sp
from scipy import ndimage
import matplotlib.pyplot as plt 
from matplotlib.colors import Normalize
import shapefile as sf
from scipy.interpolate import RegularGridInterpolator
from gnam.model.gridmod3d import gridmod3d as gm
from gnam.model.bbox import bbox as bb
from shapely.geometry import Point, Polygon

### Load numpy array from NAM model

In [ ]:
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
#ifilename = './rect_gron_model_full_z10_props.npz'
ifilename = './z10m_nam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz

### Create Gridded Model 

In [ ]:
nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d.shape:',gm3d.shape)



### Plot subsurface slice

In [ ]:
#Slice the z=0 depth slice from gm3d
surf = gm3d.depthValsSliceFromZIndex(60)[3]
print('surf.shape',surf.shape)

#get x,y,z coordinates
xc = gm3d.getLocalCoordsPointsX() + xmin
yc = gm3d.getLocalCoordsPointsY() + ymin
zc = gm3d.getLocalCoordsPointsY() + zmin

print('xc.shape:\n',xc.shape)
print('yc.shape:\n',yc.shape)
print('zc.shape:\n',zc.shape)

#get xy coordinate pairs for ploting
xyc = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])

#get min max to normalize surface
#vp_min = np.min(surf)
#vp_min = 50.0
#vp_max = np.max(surf)
#print('vp_min:',vp_min)
#print('vp_max:',vp_max)
#surf_norm = Normalize(vp_min,vp_max)

#fig, ax = plt.subplots(1,figsize=(8,8))
#ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
#ax.set_title('Full NAM Model Surface (z=0)')
#fig.colorbar()
#plt.show()

### Plot slice

In [ ]:
#get min max to normalize surface
vp_min = np.min(surf)
vp_max = np.max(surf)
print('vp_min:',vp_min)
print('vp_max:',vp_max)
surf_norm = Normalize(vp_min,vp_max)

fig, ax = plt.subplots(1,figsize=(8,8))
sc = ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Smooth Subsurface

In [ ]:
# set sigmas
z_sig = 5*(50/dz) # tested at dz=10m and was good, so assume scale by that)
y_sig = z_sig*(dz/dy)
x_sig = z_sig*(dz/dx)
sig_meters = y_sig*50
print('sigma (m):',sig_meters)

# smooth
gm3d.smoothXYZ(x_sig,y_sig,z_sig)

# get new smoothed surface
#surf = gm3d.depthValsSliceFromZIndex(200)[0]
#print('surf.shape',surf.shape)

# get new min max to normalize surface
#vp_min = np.min(surf)
#vp_max = np.max(surf)
#surf_norm = Normalize(vp_min,vp_max)

#fig, ax = plt.subplots(1,figsize=(8,8))
#ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
#ax.set_title('Full NAM Model Surface (z=0)')
#plt.show()



## Compres and pickle the smoothed model

In [ ]:
import numpy as np
st_dz = 'z' + str(int(dz)) + 'm_'
st_sig = 'sig' + str(int(sig_meters)) + 'm_'
smth_props = gm3d.getNPArray()
osfqn = './smooth_' + st_dz + st_sig + 'intnam_model_vp_vs_rho_Q_props.npz'
print(osfqn)
np.savez_compressed(osfqn,props=smth_props,xd=xdata,yd=ydata,zd=zdata)

### Unpickle smoothed model if needed

In [ ]:
#del gm3d
#del smth_props
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
ifilename = './smooth_z10m_sig250m_intnam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz

nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d.shape:',gm3d.shape)

#free up some memory
del props 

## Get subsurface depth slice of smoothed model

In [ ]:
# get new smoothed surface
surf = gm3d.depthValsSliceFromZIndex(200)[0]
print('surf.shape',surf.shape)


## Plot smoothed subsurface slice

In [ ]:
xc = gm3d.getLocalCoordsPointsX() + xmin
yc = gm3d.getLocalCoordsPointsY() + ymin
zc = gm3d.getLocalCoordsPointsY() + zmin

print('xc.shape:\n',xc.shape)
print('yc.shape:\n',yc.shape)
print('zc.shape:\n',zc.shape)

#get xy coordinate pairs for ploting
xyc = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])

# get new min max to normalize surface
vp_min = np.min(surf)
vp_max = np.max(surf)
surf_norm = Normalize(vp_min,vp_max)

fig, ax = plt.subplots(1,figsize=(8,8))
sc = ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
ax.set_title('Full NAM Model Surface (z=0)')
fig.colorbar(sc)
plt.show()

### Subsample model to reduce memory foot print

In [ ]:
#gm3d.subsample(2,2,10)
sub_dz = 10
gm3d.subsample(isz=5,idz=sub_dz) # idx=idy=2 by default
print('gm3d:',gm3d)

## Compres and pickle the subsampled model

In [ ]:
import numpy as np
sub_props = gm3d.getNPArray()
#ossfqn = './subsamp_smooth_z' + str(dz) + 'm_nam_model_vp_vs_rho_Q_props.npz'
ossfqn = './subsamp_smooth_z' + str(int(dz*sub_dz)) + 'm_nam_model_vp_vs_rho_Q_props.npz'
print(ossfqn)
print('xdata:',xdata)
print('ydata:',ydata)
print('zdata:',zdata)

xdata[0] = gm3d.get_gorigin()[0]
ydata[0] = gm3d.get_gorigin()[1]
zdata[0] = gm3d.get_gorigin()[2]
xdata[1] = gm3d.get_deltas()[0]
ydata[1] = gm3d.get_deltas()[1]
zdata[1] = gm3d.get_deltas()[2]
xdata[2] = gm3d.get_npoints()[0]
ydata[2] = gm3d.get_npoints()[1]
zdata[2] = gm3d.get_npoints()[2]
print('xdata:',xdata)
print('ydata:',ydata)
print('zdata:',zdata)
np.savez_compressed(ossfqn,props=sub_props,xd=xdata,yd=ydata,zd=zdata)

### Unpickle subsampled model if needed

In [ ]:
#this is a pickled dictionary with 4D ndarray, and 1D meta data arrays
#ifilename = './subsamp_smooth_z10.0m_nam_model_vp_vs_rho_Q_props.npz'
ifilename = './subsamp_smooth_z' + str(int(dz*sub_dz)) + 'm_nam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
props = data['props'] #4D ndarray

#meta data arrays
xdata = data['xd'] 
ydata = data['yd']
zdata = data['zd']

print('xd:\n',xdata)
print('yd:\n',ydata)
print('zd:\n',zdata)

# Setup Coordinate related vars
xmin = xdata[0]
dx   = xdata[1]
nx   = int(xdata[2])
xmax = xmin + (nx-1)*dx

ymin = ydata[0]
dy   = ydata[1]
ny   = int(ydata[2])
ymax = ymin + (ny-1)*dy

zmin = zdata[0]
dz   = zdata[1]
nz   = int(zdata[2])
zmax = (-zmin) + (nz-1)*dz

nsub_props = props.shape[0]
axes_order = {'X':0,'Y':1,'Z':2} #this dict keeps track of axes order
gm3d = gm(props,nsub_props,axes_order,(nx,ny,nz),(dx,dy,dz),(xmin,ymin,zmin))
print('gm3d.shape:',gm3d.shape)

#free up some memory
del props 

### Setup coordinate arryas for plotting and slicing

In [ ]:
xc = gm3d.getLocalCoordsPointsX() + xmin
yc = gm3d.getLocalCoordsPointsY() + ymin
zc = gm3d.getLocalCoordsPointsY() + zmin

# get new xy coordinate pairs for ploting
xyc = np.transpose([np.tile(xc, len(yc)), np.repeat(yc, len(xc))])

print('xc.shape:\n',xc.shape)
print('yc.shape:\n',yc.shape)
print('zc.shape:\n',zc.shape)

# get new smoothed surface
surf = gm3d.depthValsSliceFromZIndex(20)[0]
print('surf.shape',surf.shape)

# get new min max to normalize surface
vp_min = np.min(surf)
vp_max = np.max(surf)
surf_norm = Normalize(vp_min,vp_max)

fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm)
ax.set_title('Full NAM Model Surface (z=0)')
plt.show()


### Read seismometer coordinates file and process for coordinates

In [ ]:
import pandas as pd
import numpy as np

# read file
df = pd.io.parsers.read_csv("Gloc.csv",sep=",",index_col=0)
print(df)
print()

# remove Lat and Lon and surf elevation
df = df.drop(columns=['Latitude [deg]', 'Longitude [deg]', 'Surface elevation [m]'])
print('Dropped Lat, Lon, Surf')
print(df)
print()

# remove receivers unless depth is 200 meters
df = df[df['Depth below surface [deg]'] > 100 ]
print('Remove Shallow')
print(df)
print()

# drop this column
df = df.drop(columns=['Depth below surface [deg]'])
print('Dropped Depth')
print(df[:10])
print()

#get rec_x
rec_x = df[['Rijksdriehoek X [m]']].to_numpy().astype(np.float32)
#reshape to vector of only one dim
rec_x = rec_x.reshape(rec_x.shape[0])

#get rec_y
rec_y = df[['Rijksdriehoek Y [m]']].to_numpy().astype(np.float32)
#reshape to vector of only one dim
rec_y = rec_y.reshape(rec_y.shape[0])

### Plot overlay of seismometer locations

In [ ]:
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=1)
ax.set_title('Full NAM Model Surface w/ Field Shape')
plt.show()

### Read Groningen Shape file

In [ ]:
mysf = sf.Reader('FieldShapeFile/Groningen_field')
print('mysf:',mysf)
print('mysf.shapes():',mysf.shapes())
s = mysf.shape(0)

### Get Coordinates for Groningen Field Shape and plot

In [ ]:
mypoints = np.asarray(s.points)
print(mypoints)

fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=1)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=2)
ax.set_title('Full NAM Model Surface w/ Field Shape')
plt.show()

### Get and plot bounding box of the ShapeFile

In [ ]:
mybbox = s.bbox #this will be used for slicing (look further down)
print('mybbox:',mybbox)

pbbox_x = np.array([mybbox[0],mybbox[0],mybbox[2],mybbox[2],mybbox[0]])
pbbox_y = np.array([mybbox[1],mybbox[3],mybbox[3],mybbox[1],mybbox[1]])

c_loop = np.array([[mybbox[0],mybbox[1]],[mybbox[0],mybbox[3]],
                   [mybbox[2],mybbox[3]],[mybbox[2],mybbox[1]],
                   [mybbox[0],mybbox[1]]])

gf_bbox = bb(c_loop)

fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=1)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=2)
ax.plot(pbbox_x,pbbox_y,c='black',zorder=3)
ax.plot(gf_bbox.getCLoop()[:,0],gf_bbox.getCLoop()[:,1],c='yellow',linestyle='dotted',zorder=3)
ax.set_title('NAM Model w/ Field Shape and Bbox')
plt.show()

### Define function for separating receivers that are in and out of the bounding box

In [ ]:
def get_in_out_recs(bbox_x,bbox_y,_rec_x,_rec_y):
    xycoords = np.append(_rec_x,_rec_y).reshape((2,len(_rec_x))).T

    acoords = np.array([[bbox_x[0],bbox_y[0]],[bbox_x[1],bbox_y[1]],[bbox_x[2],bbox_y[2]],[bbox_x[3],bbox_y[3]]])
    boxcoords = list(map(tuple, acoords))
    #print('boxcoords:',boxcoords)

    poly = Polygon(boxcoords)
    #print('poly:',poly)

    xyPoints = list(map(Point, xycoords))
    is_iside = np.ones((len(xycoords[:,0])),dtype=bool)
    is_oside = np.zeros((len(xycoords[:,0])),dtype=bool)
    for i in range(len(xyPoints)):
        if not poly.contains(xyPoints[i]):
            is_iside[i] = False
            is_oside[i] = True

    #print('is_iside:\n', is_iside)

    i_stations = xycoords[is_iside]
    o_stations = xycoords[is_oside]
    
    return (i_stations,o_stations)

#irec, orec = get_in_out_recs(new_box_x,new_box_y,rec_x,rec_y)


### Plot NAM plus Field plus Bounding Box plus IN and OUT Seismometers

In [ ]:
irec, orec = get_in_out_recs(pbbox_x,pbbox_y,rec_x,rec_y)
bbirec, bborec = gf_bbox.separateByInOut(rec_x,rec_y)

fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=1)
ax.scatter(irec[:,0],irec[:,1],s=50,c='yellow',marker='v',zorder=1)
ax.scatter(orec[:,0],orec[:,1],s=60,c='black',marker='x',zorder=1)
ax.scatter(bbirec[:,0],bbirec[:,1],s=30,c='black',marker='.',zorder=1)
ax.scatter(bborec[:,0],bborec[:,1],s=30,c='yellow',marker='.',zorder=1)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=2)
ax.plot(pbbox_x,pbbox_y,c='yellow',zorder=3)
ax.set_title('NAM Model w/ Field Shape and Bbox')
plt.show()

### Shrink the bounding box for computational reasons and then create coordinates

In [ ]:
#shrink and create y coordinates for slicing box
vl = np.array([0,0.87*(mybbox[3]-mybbox[1])])
dvl = ((0.8575*(mybbox[3]-mybbox[1]))**2)**0.5
nvl = dvl//100 + 1
y = np.arange(nvl)*100
print('nvl:',nvl)

#shrink and create x coordinates for slicing box
vb = np.array([0.85*(mybbox[2]-mybbox[0]),0])
dvb = ((0.8625*(mybbox[2]-mybbox[0]))**2)**0.5
nvb = dvb//100 + 1
x = np.arange(nvb)*100
print('nvb:',nvb)

#create set of xy coordinates for slicing box
xy = np.transpose([np.tile(x, len(y)), np.repeat(y, len(x))])
print('xy.shape:',xy.shape)

#global origins
xy_xmin = np.min(x)
xy_xmax = np.max(x)
xy_ymin = np.min(y)
xy_ymax = np.max(y)

shnk_bbox_x = np.array([xy_xmin,xy_xmin,xy_xmax,xy_xmax,xy_xmin])
shnk_bbox_y = np.array([xy_ymin,xy_ymax,xy_ymax,xy_ymin,xy_ymin])
shnk_bbox_x += mybbox[0] #translate to global
shnk_bbox_y += mybbox[1] #translate to global

sc_loop = np.array([[xy_xmin,xy_ymin],[xy_xmin,xy_ymax],
                   [xy_xmax,xy_ymax],[xy_xmax,xy_ymin],
                   [xy_xmin,xy_ymin]])

sgf_bbox = bb(sc_loop)
sgf_bbox.translate(mybbox[0],mybbox[1])
print('Rotation:',sgf_bbox.getRotDeg())

irec, orec = get_in_out_recs(shnk_bbox_x,shnk_bbox_y,rec_x,rec_y)
bbirec, bborec = sgf_bbox.separateByInOut(rec_x,rec_y)

fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=1)
ax.scatter(irec[:,0],irec[:,1],s=50,c='yellow',marker='v',zorder=1)
ax.scatter(orec[:,0],orec[:,1],s=60,c='black',marker='x',zorder=1)
ax.scatter(bbirec[:,0],bbirec[:,1],s=30,c='black',marker='.',zorder=2)
ax.scatter(bborec[:,0],bborec[:,1],s=30,c='yellow',marker='.',zorder=2)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=2)
ax.plot(pbbox_x,pbbox_y,c='black',zorder=3)
ax.plot(shnk_bbox_x,shnk_bbox_y,c='yellow',zorder=4)
ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='black',linestyle='dotted',zorder=4)
ax.set_title('NAM Model w/ Field Shape and Bbox')
plt.show()


### Rotate the bounding box 

In [ ]:
#setup rotation matrices
degree = 30
theta = degree*np.pi/180
rm = np.array([[np.cos(theta),-np.sin(theta)],[np.sin(theta),np.cos(theta)]])
rmi = np.array([[np.cos(-theta),-np.sin(-theta)],[np.sin(-theta),np.cos(-theta)]])

#non rotated bounding box
rot_bx = np.array([xy_xmin,xy_xmin,xy_xmax,xy_xmax,xy_xmin])
rot_by = np.array([xy_ymin,xy_ymax,xy_ymax,xy_ymin,xy_ymin])

#yet to be rotated bounding box
rbbox_x = np.zeros_like(rot_bx)
rbbox_y = np.zeros_like(rot_by)

#rotate bounding box
rbbox_x[0] = rm.dot(np.array([rot_bx[0],rot_by[0]]))[0]
rbbox_y[0] = rm.dot(np.array([rot_bx[0],rot_by[0]]))[1]
rbbox_x[1] = rm.dot(np.array([rot_bx[1],rot_by[1]]))[0]
rbbox_y[1] = rm.dot(np.array([rot_bx[1],rot_by[1]]))[1]
rbbox_x[2] = rm.dot(np.array([rot_bx[2],rot_by[2]]))[0]
rbbox_y[2] = rm.dot(np.array([rot_bx[2],rot_by[2]]))[1]
rbbox_x[3] = rm.dot(np.array([rot_bx[3],rot_by[3]]))[0]
rbbox_y[3] = rm.dot(np.array([rot_bx[3],rot_by[3]]))[1]
rbbox_x[4] = rbbox_x[0]
rbbox_y[4] = rbbox_y[0]

#rotate coordinates
for i in range(len(xy[:,0])):
    xy[i,:] = rm.dot(xy[i,:])
    
#move box to global coordinate location to cover the Groningen Field (this was done by eye)
#play wiht the coordinate translations to movie bounding box
prbbox_x = np.copy(rbbox_x)
prbbox_y = np.copy(rbbox_y)
prbbox_x += mybbox[0]
prbbox_y += mybbox[1]

irec, orec = get_in_out_recs(prbbox_x,prbbox_y,rec_x,rec_y)

sgf_bbox.rotate(30)
bbirec, bborec = sgf_bbox.separateByInOut(rec_x,rec_y)
print('Rotation:',sgf_bbox.getRotDeg())

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=1)
ax.scatter(irec[:,0],irec[:,1],s=50,c='yellow',marker='v',zorder=1)
ax.scatter(orec[:,0],orec[:,1],s=60,c='black',marker='x',zorder=1)
ax.scatter(bbirec[:,0],bbirec[:,1],s=30,c='black',marker='.',zorder=2)
ax.scatter(bborec[:,0],bborec[:,1],s=30,c='yellow',marker='.',zorder=2)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=2)
ax.plot(shnk_bbox_x,shnk_bbox_y,c='black',zorder=3)
ax.plot(prbbox_x,prbbox_y,c='yellow',zorder=4)
ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='black',linestyle='dotted',zorder=4)
ax.set_title('NAM Model w/ Field Shape and Bbox')
plt.show()



### Translate rotated bbox

In [ ]:
import copy
#move box to global coordinate location to cover the Groningen Field (this was done by eye)
#play wiht the coordinate translations to movie bounding box
#xshift = 12600  #original?
xshift = 12300  #original?
yshift = -2600 #original?
#yshift = -2700 
#xshift = 12800
#yshift = -3500
old_box_x = np.copy(prbbox_x)
old_box_y = np.copy(prbbox_y)
new_box_x = np.copy(prbbox_x)
new_box_y = np.copy(prbbox_y)
new_box_x += xshift
new_box_y += yshift

irec, orec = get_in_out_recs(new_box_x,new_box_y,rec_x,rec_y)
cpy_bbox = copy.deepcopy(sgf_bbox)
sgf_bbox.translate(xshift,yshift)
bbirec, bborec = sgf_bbox.separateByInOut(rec_x,rec_y)

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=1)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=2)
ax.scatter(irec[:,0],irec[:,1],s=50,c='yellow',marker='v',zorder=1)
ax.scatter(orec[:,0],orec[:,1],s=60,c='black',marker='x',zorder=1)
ax.scatter(bbirec[:,0],bbirec[:,1],s=30,c='black',marker='.',zorder=2)
ax.scatter(bborec[:,0],bborec[:,1],s=30,c='yellow',marker='.',zorder=2)
ax.plot(old_box_x,old_box_y,c='black',zorder=3)
ax.plot(new_box_x,new_box_y,c='yellow',zorder=3)
ax.plot(sgf_bbox.getCLoop()[:,0],sgf_bbox.getCLoop()[:,1],c='black',linestyle='dotted',zorder=4)
ax.set_title('Overlay of Bounding Box for Slicing')
plt.show()
#tmp_bbox = sgf_bbox
#sgf_bbox = cpy_bbox
#del tmp_bbox

## Now we have the region we want to slice. 

In [ ]:
# need to translate the coordinates for interpolation to the coorect location
rxy = np.copy(xy)
rxy[:,0] += mybbox[0] + xshift
rxy[:,1] += mybbox[1] + yshift

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
ax.fill(new_box_x,new_box_y,c='white',zorder=1)
#ax.scatter(xy[:,0],xy[:,1],s=1,c=slice_surf,cmap=plt.cm.jet,norm=mynorm_slice,zorder=1)
#ax.scatter(rxy[:,0],rxy[:,1],s=1,c='white',zorder=1)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=2)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=2)
ax.scatter(irec[:,0],irec[:,1],s=50,c='black',marker='v',zorder=3)
ax.plot(new_box_x,new_box_y,c='black',zorder=4)
ax.set_title('Overlay of Bounding Box for Slicing')
plt.show()

### Interpolate and slice (make a tile) of a rotated depth slice 

In [ ]:
%matplotlib notebook

import pickle
#slice via xy coords and gm3d
slice_surf = gm3d.depthValsSliceFromXYCoordsZIndex(rxy,20,local=False)[0]

#Ploting normalization
vp_min_s = np.min(slice_surf)
vp_max_s = np.max(slice_surf)
mynorm_slice = Normalize(vp_min_s,vp_max_s)

#plot background NAM as white
bgbox_x = np.array([xc[0],xc[0],xc[-1],xc[-1],xc[0]])
bgbox_y = np.array([yc[0],yc[-1],yc[-1],yc[0],yc[0]])

print('bbox before pickle:\n',sgf_bbox)
f = open('./model_bbox.pickle', 'wb')
pickle.dump(sgf_bbox, f)
f.close()

f = open('./model_bbox.pickle', 'rb')
dill_bbox = pickle.load(f)
f.close()
print('bbox after pickle :\n',dill_bbox)

#Plot
fig, ax = plt.subplots(1,figsize=(8,8))
ax.fill(bgbox_x,bgbox_y,c='white',zorder=0)
sc = ax.scatter(rxy[:,0],rxy[:,1],s=1,c=slice_surf,cmap=plt.cm.jet,norm=mynorm_slice,zorder=1)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=2)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=2)
ax.scatter(irec[:,0],irec[:,1],s=50,c='black',marker='v',zorder=3)
ax.plot(new_box_x,new_box_y,c='black',zorder=4)
ax.set_title('Overlay of Bounding Box for Slicing')
fig.colorbar(sc)
plt.show()

### We put it all together.

In [ ]:
%matplotlib notebook
#Plot
fig, ax = plt.subplots(1,figsize=(5,5))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=surf_norm,zorder=0)
ax.fill(new_box_x,new_box_y,c='white',zorder=1)
#ax.fill(bgbox_x,bgbox_y,c='white',zorder=1)
ax.scatter(rxy[:,0],rxy[:,1],s=1,c=slice_surf,cmap=plt.cm.jet,norm=surf_norm,zorder=2)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=3)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=3)
ax.scatter(irec[:,0],irec[:,1],s=50,c='black',marker='v',zorder=4)
ax.plot(new_box_x,new_box_y,c='black',zorder=5)
ax.set_title('Overlay of Bounding Box for Slicing')
plt.ion()
plt.show()

### Interpolate Subsurface Volume

In [ ]:
import time

#get sliced subsurface volume
start = time.time()
slice_props = gm3d.sliceVolumeValsFromCoordsXY(x,y,rxy,local=False)
end = time.time()

print('runtime:', end - start)

### Compress and pickle sliced Volume

In [ ]:
print('len(x):',len(x))
print('len(y):',len(y))
# compress and pickle new volume
orrssslfqn = './rect_rot_subsamp_smooth_z' + str(int(sub_dz*dz)) + 'm_nam_model_vp_vs_rho_Q_props.npz'
print(orrssslfqn)
np.savez_compressed(orrssslfqn,props=slice_props,xc=x,yc=y,rxyc=rxy)

In [ ]:
slice_props[3,100,100,0]

### Unpickle the more sparse sliced volume

In [ ]:
ifilename = './rect_rot_subsamp_smooth_z200.0m_nam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
slice_props2 = data['props'] #4D ndarray
xc2=data['xc']
yc2=data['yc']
rsxy2=data['rxyc']

### Unpickle the less sparse sliced volume

In [ ]:
ifilename = './rect_rot_subsamp_smooth_z10m_nam_model_vp_vs_rho_Q_props.npz'

#Unpickle
data = np.load(ifilename)
slice_props = data['props'] #4D ndarray
xc=data['xc']
yc=data['yc']
rsxy=data['rxyc']

print('slice_props.shape:',slice_props.shape)

In [ ]:
sprops = np.copy(slice_props.reshape((4,61, 386, 291)),order='C')
np.min(sprops[2,0,:,:])
#rdep_surf = sprops[0,20,:,:].copy()
#sprops2 = np.copy(slice_props2.reshape((4,31, 193, 146)),order='C')
#rdep_surf2 = sprops2[0,10,:,:].copy()

In [ ]:
#%matplotlib notebook
#sprops = np.copy(slice_props.reshape((4,61, 386, 291)),order='C')
#rdep_surf = sprops[0,20,:,:].copy()
#sprops2 = np.copy(slice_props2.reshape((4,31, 193, 146)),order='C')
#rdep_surf2 = sprops2[0,10,:,:].copy()

#print('is C contig:',sprops[0,:,:,:].flatten().flags['C_CONTIGUOUS'])
#print('is C contig:',sprops[0,:,:,:].flatten().flags['F_CONTIGUOUS'])

#Plot
fig, ax = plt.subplots(1,figsize=(6,6))
ax.scatter(xyc[:,0],xyc[:,1],s=1,c=surf.flatten(),cmap=plt.cm.jet,norm=mynorm_slice,zorder=0)
ax.fill(bgbox_x,bgbox_y,c='white',zorder=1)
#ax.fill(new_box_x,new_box_y,c='white',zorder=1)
#ax.scatter(rxy[:,0],rxy[:,1],s=1,c=slice_surf,cmap=plt.cm.jet,norm=mynorm_slice,zorder=2)
#ax.scatter(rxy[:,0],rxy[:,1],s=1,c='black',zorder=2)
#ax.scatter(rsxy2[:,0],rsxy2[:,1],s=1,c=rdep_surf2.flatten(),cmap=plt.cm.jet,norm=mynorm_slice,zorder=2)
ax.scatter(rsxy2[:,0],rsxy2[:,1],s=1,c='white',zorder=3)
ax.scatter(rsxy[:,0],rsxy[:,1],s=1,c=rdep_surf.flatten(),cmap=plt.cm.jet,norm=mynorm_slice,zorder=2)
#ax.scatter(rsxy[:,0],rsxy[:,1],s=1,c='white',zorder=2)
ax.scatter(mypoints[:,0],mypoints[:,1],s=1,c='black',zorder=4)
#ax.scatter(rec_x,rec_y,s=50,c='black',marker='v',zorder=4)
ax.scatter(irec[:,0],irec[:,1],s=50,c='black',marker='v',zorder=5)
#ax.plot(new_box_x,new_box_y,c='black',zorder=6)
ax.set_title('Overlay of Bounding Box for Slicing')
plt.ion()
plt.show()

### Write VTK file

In [ ]:
from gnam.vtkutils.write import write_vtk_gridded_model_3d

rs_props = sprops.transpose(0,3,2,1).copy()
print('rs_props.shape:',rs_props.shape)
rxdata = np.zeros((3))
rydata = np.zeros((3))
rzdata = np.zeros((3))
rxdata[1] = gm3d.get_deltas()[0]
rydata[1] = gm3d.get_deltas()[1]
rzdata[1] = gm3d.get_deltas()[2]
rxdata[2] = rs_props.shape[1]
rydata[2] = rs_props.shape[2]
rzdata[2] = rs_props.shape[3]
print('rxdata:',rxdata)
print('rydata:',rydata)
print('rzdata:',rzdata)
vtkfqpname = './rect_rot_wQ_z100m'
print('vtkfqpname:',vtkfqpname)
write_vtk_gridded_model_3d(vtkfqpname,rs_props,rxdata,rydata,rzdata)